# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [9]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-06c22cdbe2-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

## Task 1
Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

In [10]:
#STRATEGY: 
#For all the unknown nodes that I want to explore for the first time I will follow the following strategy: 
# - (1) explore all the outcoming object properties from the node and I will return the property IRI, name and the related object IRI and name 
# - (2) explore all the incoming object properties from the node and I will return the property IRI, name and the related object IRI and name 
# - (3) explore all the node data properties

#When I do one of the previous explorative queries, I will write: "Explore **node** by performing (1)" (1) is for the query that explore all the outcoming object properties ... and so on

#I will not do all the previous steps every time, if I find what I need, I will "stop" the exploration strategy

#NOTICE THAT:

#If I have to retrieve the BGP of a node I will do a simple query that selects only the specified node with the 
#specified URI and I retrieve also the label. Obviously the URI that I used is retrieved in the queries that were done
#before that one.


In [11]:
#explore the Arthur Conan Doyle node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q35610 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('oName', 'Charles Altamont Doyle')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion'), ('oName', 'Lapsed Catholic')]
[('p', 'http://www.wikidata.org/prop/direct/P172'), ('pName', 'ethnic group'), ('oName', 'Irish people')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('oName', 'crime novel')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('oName', "The New Student's Reference Work")]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('oName', 'detective novel')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('oName', 'cricket')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('oName', 'Stonyhurst College')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pName', 'notable work'), ('oName', 'canon of Sherlock Holmes')]
[('p', '

In [12]:
#explore the initial node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q35610 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q47183632'), ('sName', 'The Baker Street Irregulars'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q4798323'), ('sName', 'Arthur Conan Doyle bibliography'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of')]
[('s', 'http://www.wikidata.org/entity/Q253545'), ('sName', '7016 Conandoyle'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('s', 'http://www.wikidata.org/entity/Q96102817'), ('sName', 'out-of-universe perspective'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('s', 'http://www.wikidata.org/entity/Q47490'), ('sName', "Doyle's Delight"), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('s', 'http://www.wikidata.org/entity/Q22689379'), ('sName', 'Conan Edogawa'), ('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('s',

In [13]:
#select from the results of (1) every object related to Sherlock holmes

queryString = """
SELECT DISTINCT ?p ?nameProp ?obj ?nameObj
WHERE {
   wd:Q35610 ?p ?obj .
   ?p sc:name ?nameProp.
   ?obj sc:name ?nameObj. 
   
   FILTER (REGEX(?nameObj, "[S,s]herlock"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('nameProp', 'notable work'), ('obj', 'http://www.wikidata.org/entity/Q2316684'), ('nameObj', 'canon of Sherlock Holmes')]
1


Final query for this task

In [14]:
# explore the previously returned URI 

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q2316684)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('uri', 'http://www.wikidata.org/entity/Q2316684'), ('name', 'canon of Sherlock Holmes')]
1


## Task 2
Identify the BGP for film

In [15]:
#explore the 'canon of Sherlock Holmes' node by performing (1) 

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q2316684 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('pName', 'derivative work'), ('obj', 'http://www.wikidata.org/entity/Q1764019'), ('oName', 'Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q733582'), ('oName', 'His Last Bow')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q45192'), ('oName', 'The Hound of the Baskervilles')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q277759'), ('oName', 'book series')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q860831'), ('oName', 'The Memoirs of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1631107'), ('oName', 'bibliography')]
[('p', 'http://www.wikid

In [16]:
#explore the 'canon of Sherlock Holmes' node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q2316684 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q127149'), ('sName', 'Lolita'), ('p', 'http://www.wikidata.org/prop/direct/P8371'), ('pName', 'references work, tradition or theory')]
[('s', 'http://www.wikidata.org/entity/Q47183632'), ('sName', 'The Baker Street Irregulars'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q1764019'), ('sName', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q192837'), ('sName', 'Sherlock'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q4727821'), ('sName', 'All-Consuming Fire'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q70734078'), ('sName', 'Sherlock Holmes and the Time machine'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', '

In [17]:
#from the results obtained in the step (1) of the exploration of the Canon of Sherlock Holmes node, explore one of the object returned (Q733582) with the has part property

#explore it by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q733582 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pName', 'part of the series'), ('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oName', 'canon of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q20665028'), ('oName', 'detective literature')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pName', 'language of work or name'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P6216'), ('pName', 'copyright status'), ('obj', 'http://www.wikidata.org/entity/Q19652'), ('oName', 'public domain')]
[('p', 'http://www.wikidata.org/prop/direct/P840'), ('pName', 'narrative location'), ('obj', 'http://www.wikidata.org/entity/Q84'), ('oName', 'London')]
[('p', 'http://www

In [18]:
#the previous node is an instance of short story collection, so maybe it's not what I'm looking for

#from the results obtained in the step (1) of the canon of Sherlock Holmes node exploration, explore one of the object returned with the derivative work property (Q1764019)

#explore it by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q1764019 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q24856'), ('oName', 'film series')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on'), ('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oName', 'canon of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pName', 'original language of film or TV show'), ('obj', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P462'), ('pName', 'color'), ('obj', 'http://www.wikidata.org/entity/Q838368'), ('oName', 'black and white')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q728050'), ('oName', 'Nigel Bruce')]
[('p', 'http://www.wi

In [19]:
#the previous object is an instance of film series, so maybe I'm in the rigth way
#explore the Q24856 film series object (instance of the Q1764019 node just explored) by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q24856 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q17123180'), ('oName', 'sequel film')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q11424'), ('oName', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q196600'), ('oName', 'media franchise')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q838948'), ('oName', 'work of art')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2431196'), ('oName', 'audiovisual work')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q7751682'), ('oName', 'serial film')]
[('p', 'http://www.wikidata.org/prop/direct/

In [20]:
#explore the film node (Q11424) returned in the previous query with the "has parts of the class" property by performing (1)  

queryString = """
SELECT DISTINCT ?p ?nameProp ?obj ?nameObj
WHERE {
   wd:Q11424 ?p ?obj .
   ?p sc:name ?nameProp .
   ?obj sc:name ?nameObj
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('nameProp', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q1660187'), ('nameObj', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P345'), ('nameObj', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1040'), ('nameObj', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P344'), ('nameObj', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('nameProp', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q17738'), ('nameObj', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('nameProp', "topic's main category"), ('obj', 'http://www.wikidata.org/en

In [21]:
#retrieve the BGP of the film node 

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q11424)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('uri', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 3
Identify the BGP for television series

In [22]:
#start from the previous results

#explore the Q24856 film series object previously retrieved and all the related properties and objects

queryString = """
SELECT DISTINCT ?obj ?nameProp ?nameObj
WHERE {
   wd:Q24856 ?p ?obj .
   ?p sc:name ?nameProp .
   ?obj sc:name ?nameObj
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q17123180'), ('nameProp', 'has part'), ('nameObj', 'sequel film')]
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('nameProp', 'has parts of the class'), ('nameObj', 'film')]
[('obj', 'http://www.wikidata.org/entity/Q196600'), ('nameProp', 'subclass of'), ('nameObj', 'media franchise')]
[('obj', 'http://www.wikidata.org/entity/Q838948'), ('nameProp', 'subclass of'), ('nameObj', 'work of art')]
[('obj', 'http://www.wikidata.org/entity/Q2431196'), ('nameProp', 'subclass of'), ('nameObj', 'audiovisual work')]
[('obj', 'http://www.wikidata.org/entity/Q7751682'), ('nameProp', 'different from'), ('nameObj', 'serial film')]
[('obj', 'http://www.wikidata.org/entity/Q7725310'), ('nameProp', 'subclass of'), ('nameObj', 'series of creative works')]
[('obj', 'http://www.wikidata.org/entity/Q33111614'), ('nameProp', 'has part'), ('nameObj', 'prequel film')]
[('obj', 'http://www.wikidata.org/entity/Q7237133'), ('nameProp', "topic's main category")

In [23]:
#from the previous results I don't find something useful

#explore again all the nodes related to the Canon of Sherlock Holmes node with the derivative work property

queryString = """
SELECT DISTINCT ?obj ?nameObj ?p ?pname ?obj1name
WHERE {
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj sc:name ?nameObj . 
   ?obj ?p ?obj1 . 
   
   #get the labels
   ?p sc:name ?pname . 
   ?obj1 sc:name ?obj1name
}
LIMIT 100
"""

#this object is an instance of film series, so maybe it's what I'm looking for

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('nameObj', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('obj1name', 'James Herbert')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('nameObj', 'Sherlock Holmes: A Game of Shadows'), ('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('obj1name', 'James Herbert')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('nameObj', 'Sherlock'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj1name', 'thriller television series')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('nameObj', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj1name', 'buddy film')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('nameObj', 'Elementary'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('obj1name', 'television adaptation')]
[('obj', 'http://www.w

In [24]:
# I retrieve all the derivative works that are television series, by searching for all the nodes that are instance of 
# something similar to television series keyword

queryString = """
SELECT DISTINCT ?obj ?nameObj ?p ?pname ?obj1 ?obj1name
WHERE {
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj sc:name ?nameObj . 
   
   ?obj wdt:P31 ?obj1 . 
   
   ?obj1 sc:name ?obj1name . 
   FILTER (REGEX(?obj1name, "^television series" ))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('nameObj', 'Sherlock'), ('obj1', 'http://www.wikidata.org/entity/Q5398426'), ('obj1name', 'television series')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('nameObj', 'Elementary'), ('obj1', 'http://www.wikidata.org/entity/Q5398426'), ('obj1name', 'television series')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('nameObj', 'Sherlock Holmes'), ('obj1', 'http://www.wikidata.org/entity/Q5398426'), ('obj1name', 'television series')]
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('nameObj', 'Sherlock Holmes'), ('obj1', 'http://www.wikidata.org/entity/Q5398426'), ('obj1name', 'television series')]
4


In [25]:
#explore the television series node

queryString = """
SELECT DISTINCT ?p ?nameProp ?obj ?nameObj
WHERE {
   wd:Q5398426 ?p ?obj .
   ?p sc:name ?nameProp . 
   ?obj sc:name ?nameObj . 
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1113'), ('nameObj', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('nameProp', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q15416'), ('nameObj', 'television program')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('nameObj', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('nameObj', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('nameProp', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1476'), ('nameObj', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('nameProp', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q131758'), ('nameObj',

Final query for this task

In [26]:
#retrieve from the previous results the BGP for the television series node 

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q5398426)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('uri', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
1


## Task 4
Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

In [27]:
#explore one of the television series nodes (Q192837) previously retrieved by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q192837 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('obj', 'http://www.wikidata.org/entity/Q67175872'), ('oName', 'thriller television series')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director'), ('obj', 'http://www.wikidata.org/entity/Q14071541'), ('oName', 'Nick Hurran')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q28101242'), ('oName', 'The Six Thatchers')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q28128648'), ('oName', 'The Final Problem')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q55827477'), ('oName', 'Sherlock, season 4')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q838121'), ('oName', 'Peabody Awards')]
[('p', 'http://www.wikidata.org

In [28]:
#explore one of the television series nodes (Q192837) previously retrieved by performing (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   wd:Q192837 ?p ?obj .
   ?p sc:name ?pName.
   FILTER(isLiteral(?obj))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pName', 'number of episodes'), ('obj', '13')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID'), ('obj', 'tv/sherlock')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pName', 'AlloCiné series ID'), ('obj', '4528')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title'), ('obj', 'Sherlock')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID'), ('obj', 'v524472')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID'), ('obj', 'tv/sherlock')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('pName', 'Netflix ID'), ('obj', '70202589')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('pName', 'YouTube channel ID'), ('obj', 'UCkp_CAX1eIc5k5SavzbhN5w')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('pName', 'number of seasons'), ('obj', '4')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), 

In [29]:
#check the start time property of Q192837

queryString = """
SELECT DISTINCT ?o
WHERE {
   wd:Q192837 wdt:P580 ?o . 
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', '2010-07-25T00:00:00Z')]
1


In [30]:
#select all the film derivative works associated to the Canon Of Sherlock Holmes

queryString = """
SELECT DISTINCT ?obj ?label
WHERE {
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 wd:Q11424 ; 
        sc:name ?label .
}
LIMIT 100
"""


print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('label', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('label', 'Sherlock Holmes: A Game of Shadows')]
2


In [31]:
#check the start time of one of the film nodes

queryString = """
SELECT DISTINCT ?o
WHERE {
   wd:Q243643 wdt:P580 ?o . 
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [32]:
#the start time property cannot be used for the film so I need to explore the node of one of the film

#explore the Sherlock Holmes:Game Of Shadows node (Q243643) by perfoming (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   wd:Q243643 ?p ?obj .
   ?p sc:name ?pName.
   FILTER(isLiteral(?obj))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pName', 'Box Office Mojo film ID (former scheme)'), ('obj', 'sherlockholmes2')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pName', 'Rotten Tomatoes ID'), ('obj', 'm/sherlock_holmes_a_game_of_shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pName', 'AlloCiné film ID'), ('obj', '173048')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pName', 'title'), ('obj', 'Sherlock Holmes: A Game of Shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pName', 'AllMovie title ID'), ('obj', 'v518386')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pName', 'Metacritic ID'), ('obj', 'movie/sherlock-holmes-a-game-of-shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1804'), ('pName', 'DNF film ID'), ('obj', '75170')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('pName', 'Netflix ID'), ('obj', '70202148')]
[('p', 'http://www.wikidata.org/prop/direct/P1970'), ('pName', 'Mo

In [33]:
#check the pubblication date of the Sherlock Holmes:Game Of Shadows node (Q243643)

queryString = """
SELECT DISTINCT ?o
WHERE {
   wd:Q243643 wdt:P577 ?o . 
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', '2011-12-16T00:00:00Z')]
1


Final query for this task

In [34]:
queryString = """
SELECT DISTINCT ?obj ?name ?year
WHERE {
   #retrieve all the related works that are film or tv series
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?instance .
   FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .
   
   #get labels 
   ?obj sc:name ?name .
   
   #get pubblication date
   ?obj wdt:P580|wdt:P577 ?date .
   
   #binding of the year
   BIND(YEAR(?date) as ?year) . 
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('year', '2013')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('year', '2014')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('year', '2009')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('year', '2010')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('year', '2011')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('year', '2010')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('year', '2012')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('year', '2013')]
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('year', '1984')]
9


In [35]:
#I have to take only the first pubblication date for every node 

queryString = """
SELECT DISTINCT ?obj as ?film_or_tvSeries_IRI ?name MIN(?year) as ?year
WHERE {
   #retrieve all the related works that are film or tv series
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?instance .
   FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .
   
   #get labels 
   ?obj sc:name ?name .
   
   #get pubblication date
   ?obj wdt:P580|wdt:P577 ?date .
   
   #binding of the year
   BIND(YEAR(?date) as ?year) . 
}
GROUP BY ?obj ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film_or_tvSeries_IRI', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('year', '2013')]
[('film_or_tvSeries_IRI', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('year', '2009')]
[('film_or_tvSeries_IRI', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('year', '2012')]
[('film_or_tvSeries_IRI', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('year', '2010')]
[('film_or_tvSeries_IRI', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('year', '2011')]
[('film_or_tvSeries_IRI', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('year', '1984')]
6


## Task 5
Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

In [36]:
#explore one of the node related to the Sherlock TV series (Q192837) with the cast member relation (P161)
#explore for example the node of Mark Gatiss (Q560286) previously retrieved during the exploration phase (1) of the Sherlock Holmes tv series exploration

#explore the node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q560286 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('obj', 'http://www.wikidata.org/entity/Q7243506'), ('oName', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q10798782'), ('oName', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q10800557'), ('oName', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pName', 'given name'), ('obj', 'http://www.wikidata.org/entity/Q13610143'), ('oName', 'Mark')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pName', 'country of citizenship'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed'), ('obj', 'http://www.wikidata.org/entity/Q1860'), 

In [37]:
#explore the node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q560286 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q21527601'), ('sName', 'Denial'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q621658'), ('sName', 'Appointment with Death'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q1346200'), ('sName', 'The Reichenbach Fall'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q192837'), ('sName', 'Sherlock'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q2718184'), ('sName', 'The Wedding of River Song'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/entity/Q3203418'), ('sName', 'The Lazarus Experiment'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member')]
[('s', 'http://www.wikidata.org/

In [38]:
#check that a film node previously retrieved (Q243643) has the cast member object property (the same used in the Television Series node)

#explore the node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q243643 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q1102002'), ('oName', 'Clive Russell')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q192912'), ('oName', 'Stephen Fry')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('pName', 'distributed by'), ('obj', 'http://www.wikidata.org/entity/Q126399'), ('oName', 'Warner Bros.')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pName', 'cast member'), ('obj', 'http://www.wikidata.org/entity/Q160432'), ('oName', 'Jude Law')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'), ('pName', 'production designer'), ('obj', 'http://www.wikidata.org/entity/Q2224360'), ('oName', 'Sarah Greenwood')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on'), ('obj', 'http://www.wikidata.org/entity/Q2316684'), ('oName', 'canon of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/p

Final query for this task

In [39]:
# write your final query

queryString = """
SELECT DISTINCT ?actor ?actorName (COUNT(DISTINCT ?obj) as ?nFilms)
WHERE {
   #retrieve all the related works that are film or tv series
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?instance .
   FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .
   
   #get labels 
   ?obj sc:name ?name .
   
   #get all the actors that acted in one of the specified derivative works (films or tv series)
   ?obj wdt:P161 ?actor .
   ?actor sc:name ?actorName 
}
GROUP BY ?actor ?actorName 
HAVING(COUNT(DISTINCT ?obj) > 1)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q1102002'), ('actorName', 'Clive Russell'), ('nFilms', '2')]
[('actor', 'http://www.wikidata.org/entity/Q190386'), ('actorName', 'Rachel McAdams'), ('nFilms', '2')]
[('actor', 'http://www.wikidata.org/entity/Q45647'), ('actorName', 'Eddie Marsan'), ('nFilms', '2')]
[('actor', 'http://www.wikidata.org/entity/Q440926'), ('actorName', 'Geraldine James'), ('nFilms', '2')]
[('actor', 'http://www.wikidata.org/entity/Q165219'), ('actorName', 'Robert Downey Jr.'), ('nFilms', '2')]
[('actor', 'http://www.wikidata.org/entity/Q160432'), ('actorName', 'Jude Law'), ('nFilms', '2')]
[('actor', 'http://www.wikidata.org/entity/Q236167'), ('actorName', 'Kelly Reilly'), ('nFilms', '2')]
7


## Task 6
Identify the BGP for Primetime Emmy Award

In [40]:
#from the Mark Gatiss node exploaration (done before) I noted that there is an award related to the Primetime Emmy Award (Q7243506)
#I explore it by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q7243506 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1044427'), ('oName', 'Primetime Emmy Award')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q38033430'), ('oName', 'class of award')]
3


Final query for this task

In [41]:
#from the previous results

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q1044427)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('uri', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
1


## Task 7
Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

Final query for this task

In [42]:
#from the previous results

queryString = """
SELECT DISTINCT ?award ?awardName
WHERE {
   #select all the award that are instance of Primetime Emmy Award
   ?award wdt:P31 wd:Q1044427 . 
   ?award sc:name ?awardName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('award', 'http://www.wikidata.org/entity/Q18153303'), ('awardName', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('award', 'http://www.wikidata.org/entity/Q28445729'), ('awardName', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q989439'), ('awardName', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q2604800'), ('awardName', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q7243506'), ('awardName', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('award', 'http://www.wikidata.org/entity/Q1277961'), ('awardName', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('award', 'http://www.wikidata.org/entity/Q3123491'), ('awardName', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('

## Task 8
For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [43]:
#from the exploration done before on one of the film or tv serie nodes, I note the property P166 (award received) that can be used for 
#retrieving the awards won by a a film or tv serie

queryString = """
SELECT DISTINCT ?obj ?name ?awardName
WHERE {
   #retrieve all the related works that are film or tv series
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?instance .
   FILTER(?instance = wd:Q11424 || ?instance = wd:Q5398426) .
   
   #get labels 
   ?obj sc:name ?name .
   
   #get all the awards obtained by the film or tv serie
   ?obj wdt:P166 ?award .

   ?award sc:name ?awardName
}
GROUP BY ?obj ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awardName', 'British Academy Television Award for Best Supporting Actor')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awardName', 'Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awardName', 'British Academy Television Award for Best Drama Series')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awardName', 'Peabody Awards')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('awardName', 'Teen Choice Award for Choice TV Actress Action')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awardName', 'TCA Award for Outstanding Achievement in Movies, Miniseries and Specials')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('awardName', "Critics' 

In [44]:
#I want to filter from the previous query the award that are  instance of Emmy award

queryString = """
SELECT DISTINCT ?obj ?name (COUNT(*) as ?nAwards)
WHERE {
   #retrieve all the related works that are film or tv series
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?instance .
   FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .
   
   #get labels 
   ?obj sc:name ?name .
   
   #get all the awards obtained by the film or tv serie
   ?obj wdt:P166 ?award .
   
   #specify that the award must be an instance of Emmy award
   ?award wdt:P31 wd:Q1044427
}
GROUP BY ?obj ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('nAwards', '8')]
1


In [45]:
#I want to retrive all the films and tv series that didn't receive an award

queryString = """
SELECT DISTINCT ?obj ?name (0 as ?nAwards)
WHERE {
   #retrieve all the related works that are film or tv series
   wd:Q2316684 wdt:P4969 ?obj .
   ?obj wdt:P31 ?instance .
   FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .
   
   #get labels 
   ?obj sc:name ?name .
   
   #get all the awards obtained by the film or tv serie
   FILTER NOT EXISTS {
       ?obj wdt:P166 ?award .

       #specify that the award must be an instance of Emmy award
       ?award wdt:P31 wd:Q1044427 
   }
}
GROUP BY ?obj ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('nAwards', '0')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('nAwards', '0')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('nAwards', '0')]
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('nAwards', '0')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('nAwards', '0')]
5


Final query for this task

In [46]:
#union of the previos queries


queryString = """
SELECT * WHERE {
{

    #first query for retrieving all the films and tv series that won an Emmy
    SELECT DISTINCT (?obj as ?film_or_tvserie) ?name (COUNT(DISTINCT ?award) as ?nAwards)
    WHERE {
       #retrieve all the related works that are film or tv series
       wd:Q2316684 wdt:P4969 ?obj .
       ?obj wdt:P31 ?instance .
       FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .

       #get labels 
       ?obj sc:name ?name .

       #get all the awards obtained by the film or tv serie
       ?obj wdt:P166 ?award .

       #award is an instance of Emmy Award
       ?award wdt:P31 wd:Q1044427

    }
    GROUP BY ?obj ?name
}
UNION
{
    #first query for retrieving all the films and tv series that didn't won an Emmy
    SELECT DISTINCT (?obj as ?film_or_tvserie) ?name (0 as ?nAwards)
    WHERE {
       #retrieve all the related works that are film or tv series
       wd:Q2316684 wdt:P4969 ?obj .
       ?obj wdt:P31 ?instance .
       FILTER(?instance = wd:Q5398426 || ?instance = wd:Q11424) .

       #get labels 
       ?obj sc:name ?name .

       #filter out all the film or tv series that doesn't receive an Emmy Award
       FILTER NOT EXISTS {
           ?obj wdt:P166 ?award .

           #specify that the award must be an instance of Emmy award
           ?award wdt:P31 wd:Q1044427 
       }
    }
    GROUP BY ?obj ?name
}

}
"""

print("Results")
x=run_query(queryString)

Results
[('film_or_tvserie', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('nAwards', '8')]
[('film_or_tvserie', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('nAwards', '0')]
[('film_or_tvserie', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('nAwards', '0')]
[('film_or_tvserie', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('nAwards', '0')]
[('film_or_tvserie', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('nAwards', '0')]
[('film_or_tvserie', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('nAwards', '0')]
6
